In [1]:
import torch
import logging
import os
import io
import anvil.server
from PIL import Image
from anvil import BlobMedia
from utils import sr_utils

In [2]:
print(torch.cuda.get_device_properties(0).total_memory)
print(torch.cuda.memory_reserved(0))
print(torch.cuda.memory_allocated(0))

8589934592
0
0


In [3]:
model_name = "BSRGAN"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
logger = logging.getLogger("blind_sr_log")

In [5]:
model = sr_utils.load_model(model_name, device)

[3, 3, 64, 23, 32, 4]


In [6]:
anvil.server.connect(os.environ.get("anvil_uplink"))

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER


In [11]:
@anvil.server.callable
def image_super_resolution(image):
    # save new image
    image = Image.open(io.BytesIO(image.get_bytes()))
    image.save("testsets/diffusion/diffusion.png", format="PNG")

    # apply super resolution
    sr_utils.super_resolution(model, logger, device)
    # subprocess.run(["python", "super_resolution.py"])

    # read image as bytes
    sr_image = Image.open("results/diffusion_results/diffusion_sr.png")
    output = io.BytesIO()
    sr_image.save(output, format="PNG")
    content = output.getvalue()

    # use BlobMedia to store bytes data
    return BlobMedia("image/png", content)

In [13]:
anvil.server.wait_forever()

KeyboardInterrupt: 